In [ ]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('sentence-transformers/all-apnet-base-v2')
sentences=("this is an example sentence")
embeddings=model.encode(sentences)
print('Sentence embeddings:')
print(embeddings)

In [1]:
from PyPDF2 import PdfReader



In [ ]:
import os
from dotenv import load_dotenv
COHERE_KEY=os.environ.get("")
print("COHERE_KEY: ",COHERE_KEY)

COHERE_KEY:  ZnlMbCzCcLd0SIHx76FJ5sciIBRL8OCTbNIyYHf0


In [3]:
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS


In [4]:
embeddings=CohereEmbeddings(
    cohere_api_key=COHERE_KEY,
    model="embed-english-v3.0",  
    user_agent="langchain" )

C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3560409958.py:1: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings=CohereEmbeddings(


In [5]:
def load_pdfs(pdfs):
    text_chunks=[]
    for pdf in pdfs:
        reader=PdfReader(pdf)
        for page in reader.pages:
            raw=page.extract_text()
            chunks=split_paragraph(raw)
            text_chunks=chunks
    return text_chunks


def split_paragraph(rawText):
    text_splitter=CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=30,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_text(rawText)

In [6]:
list_of_pdfs={"1. dietary supplements - for whom.pdf"}
text_chunks=load_pdfs(list_of_pdfs)

In [7]:
text_chunks

['Int. J. Environ. Res. Public Health 2021 ,18, 8897 17 of 24\n“therapeutic” doses of nutrients should not be used in food preparations. Moreover, Polish',
 'pharmaceutical law clearly emphasizes that in the case of a product that meets the criteria\nfor classiﬁcation both as a medicinal product and as a dietary supplement, the provisions of',
 'pharmaceutical legislation, rather than food regulations, should be applied [ 147]. Currently,\nthe dose of active ingredients is an important criterion in the registration process of dietary',
 'supplements [ 148]. It is also worth remembering that labeling and advertising of dietary\nsupplements can neither imply nor make claims about the product’s preventive and/or',
 'therapeutic properties, which is consistent with food law and the regulations governing\nall foodstuffs [ 149]. Therefore, dietary supplement packaging may not contain informa-',
 'tion such as “for cancer prevention” or “for reducing cholesterol”. However, although\nsuch requ

In [9]:
store=FAISS.from_texts(text_chunks,embeddings)

In [10]:
store.save_local("./myVectorStore")

In [12]:
from langchain_cohere import ChatCohere
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import CohereEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
store=FAISS.load_local("myVectorStore",embeddings,allow_dangerous_deserialization=True)
llm = ChatCohere(
    cohere_api_key="",
    model="command-xlarge-nightly",   
    temperature=0.7)

chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=store.as_retriever()
)
result=chain({"query" : "Can consuming multi vitamin and minerals reduce my blood pressure?"}
             )
print(result)


C:\Users\HP\AppData\Local\Temp\ipykernel_19368\3604371594.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"query" : "Can consuming multi vitamin and minerals reduce my blood pressure?"}


{'query': 'Can consuming multi vitamin and minerals reduce my blood pressure?', 'result': "Based on the provided context, there is no direct information to suggest that consuming multi-vitamin and mineral supplements can reduce blood pressure. The text emphasizes the importance of interpreting research on vitamin and mineral preparations carefully, noting that the control group in studies may already have appropriate levels of these nutrients from their daily diets. It also highlights that supplements are not intended to have medicinal properties.\n\nTo determine if multi-vitamin and mineral supplements can reduce blood pressure, you would need to consult specific studies or medical research focused on this topic. If you're considering supplements for blood pressure management, it's best to consult a healthcare professional for personalized advice."}
